In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC ### 3. Movimentação: Silver para Gold
# MAGIC 
# MAGIC **Origem:** Tabela `movies_catalog.silver.movies_cleaned`
# MAGIC 
# MAGIC **Destino:** Tabelas de agregação no schema `gold`.
# MAGIC 
# MAGIC **Ações:**
# MAGIC 1. Calcula o lucro de cada filme.
# MAGIC 2. Conta a quantidade de filmes lançados por ano.
# MAGIC 3. Extrai os gêneros de uma coluna JSON e calcula a receita média para cada um.

# COMMAND ----------

from pyspark.sql.functions import col, avg, count, desc, explode, from_json
from pyspark.sql.types import ArrayType, StructType, StructField, StringType

# Garante que o catálogo correto está em uso
spark.sql("USE CATALOG movies_catalog")

# COMMAND ----------

# Carrega a tabela limpa da camada Silver
df_silver = spark.read.table("silver.movies_cleaned")

# --- AGREGAÇÃO 1: Lucro por Filme ---
df_lucro_por_filme = df_silver \
    .withColumn("lucro", col("receita") - col("orcamento")) \
    .select("titulo_original", "ano_lancamento", "orcamento", "receita", "lucro") \
    .orderBy(desc("lucro"))

df_lucro_por_filme.write.mode("overwrite").saveAsTable("gold.agg_lucro_por_filme")
print("Tabela 'gold.agg_lucro_por_filme' salva com sucesso.")

# --- AGREGAÇÃO 2: Quantidade de Filmes por Ano ---
df_filmes_por_ano = df_silver.groupBy("ano_lancamento") \
                             .agg(count("id").alias("quantidade_filmes")) \
                             .orderBy(desc("ano_lancamento"))

df_filmes_por_ano.write.mode("overwrite").saveAsTable("gold.agg_filmes_por_ano")
print("Tabela 'gold.agg_filmes_por_ano' salva com sucesso.")

# --- AGREGAÇÃO 3: Receita Média por Gênero ---
# Schema para extrair o nome do gênero do JSON aninhado
genre_schema = ArrayType(StructType([StructField("name", StringType())]))

df_generos = df_silver.withColumn("genres_array", from_json(col("genres"), genre_schema)) \
                      .withColumn("genre_struct", explode(col("genres_array"))) \
                      .withColumn("genero", col("genre_struct.name")) \
                      .select("genero", "receita")

df_receita_por_genero = df_generos.groupBy("genero") \
                                  .agg(
                                      avg("receita").alias("receita_media"),
                                      count("*").alias("quantidade_filmes")
                                  ) \
                                  .orderBy(desc("receita_media"))

df_receita_por_genero.write.mode("overwrite").saveAsTable("gold.agg_receita_media_por_genero")
print("Tabela 'gold.agg_receita_media_por_genero' salva com sucesso.")